   # Import Dependencies

In [14]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from sqlalchemy import create_engine

# Import datasets and create dataframes

In [15]:
# Reading the poverty and income csv file and creating a dataframe
income_csv_file ="2018_poverty_income.csv"
income_df = pd.read_csv(income_csv_file, encoding = "UTF-8")
income_df.rename(columns={'County Name': 'County'}, inplace=True)

#income_df

,County,Poverty Estimate,Poverty Percent,Median Household Income
0,Anderson County,"8,778",19.8,"45,969"
1,Andrews County,"1,922",10.7,"84,946"
2,Angelina County,"15,052",17.9,"46,653"
3,Aransas County,"4,642",19.9,"46,912"
4,Archer County,922,10.6,"61,190"
...,...,...,...,...
249,Wood County,"6,625",15.3,"48,384"
250,Yoakum County,"1,028",12.1,"61,560"
251,Young County,"2,468",13.9,"49,301"
252,Zapata County,"4,529",32.1,"33,160"


In [32]:
# Reading the crime csv file and creating a dataframe
crime_csv_file ="crime_data.csv"
all_crime_df = pd.read_csv(crime_csv_file, encoding = "UTF-8")

crime_df = all_crime_df.drop(columns=["Unnamed: 11", "Unnamed: 12", "Unnamed: 13", "Unnamed: 14", "Unnamed: 15", "Unnamed: 16", "Unnamed: 17", "Unnamed: 18"])

crime_df.rename(columns={'Violent\ncrime': 'Violent_Crime', 
                         'Murder and\nnonnegligent\nmanslaughter': "Murder",
                         "Rape1": "Rape", "Aggravated\nassault": "Assault",
                         "Property\ncrime": "Property_Crime",
                         "Larceny-\ntheft": "Theft",
                         "Motor\nvehicle\ntheft": "Vehicle_Theft"
                        }, inplace=True)

crime_df

,County,Violent_Crime,Murder,Rape,Robbery,Assault,Property_Crime,Burglary,Theft,Vehicle_Theft,Arson
0,Anderson County,54.0,0.0,2.0,6,46.0,350,121,188,41,0.0
1,Andrews County,17.0,0.0,2.0,0,15.0,80,18,50,12,0.0
2,Angelina County,96.0,0.0,24.0,3,69.0,492,152,274,66,1.0
3,Aransas County,76.0,0.0,18.0,3,55.0,264,94,152,18,0.0
4,Atascosa County,83.0,4.0,3.0,8,68.0,520,136,317,67,0.0
...,...,...,...,...,...,...,...,...,...,...,...
225,Zapata County,7.0,0.0,1.0,1,5.0,202,91,102,9,0.0
226,Zavala County,6.0,0.0,5.0,0,1.0,38,19,17,2,0.0
227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Reading the population csv file and creating a dataframe
population_csv_file ="population.csv"
population_df = pd.read_csv(population_csv_file, encoding = "UTF-8")

#population_df

In [34]:
# Reading the crime csv file and creating a dataframe
unemployment_csv_file ="Unemployment rate.csv"
unemployment_df = pd.read_csv(unemployment_csv_file, encoding = "UTF-8")
unemployment_df.rename(columns={'County Name': 'County'}, inplace=True)

#unemployment_df

# Merge data

In [35]:
merge_df = income_df.merge(population_df, on = "County").merge(unemployment_df, on = "County")

complete_df = merge_df.merge(crime_df, how = "left", on = "County")

complete_df

,County,Poverty Estimate,Poverty Percent,Median Household Income,Population,Unemployment Rate (%),Violent_Crime,Murder,Rape,Robbery,Assault,Property_Crime,Burglary,Theft,Vehicle_Theft,Arson
0,Anderson County,"8,778",19.8,"45,969",57735,3.1,54.0,0.0,2.0,6,46.0,350,121,188,41,0.0
1,Andrews County,"1,922",10.7,"84,946",18705,2.4,17.0,0.0,2.0,0,15.0,80,18,50,12,0.0
2,Angelina County,"15,052",17.9,"46,653",86715,4.3,96.0,0.0,24.0,3,69.0,492,152,274,66,1.0
3,Aransas County,"4,642",19.9,"46,912",23510,5.9,76.0,0.0,18.0,3,55.0,264,94,152,18,0.0
4,Archer County,922,10.6,"61,190",8553,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,Wood County,"6,625",15.3,"48,384",45539,4.2,73.0,0.0,30.0,0,43.0,201,60,117,24,0.0
250,Yoakum County,"1,028",12.1,"61,560",8713,2.9,5.0,0.0,4.0,0,1.0,30,5,21,4,0.0
251,Young County,"2,468",13.9,"49,301",18010,3.2,8.0,0.0,0.0,2,6.0,39,21,12,6,1.0
252,Zapata County,"4,529",32.1,"33,160",14179,5.5,7.0,0.0,1.0,1,5.0,202,91,102,9,0.0


# Output a combined csv file

In [36]:
#complete_df.to_csv('complete.csv')